# Aries Basic Controller - Holder Bob

This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [issuer notebook](http://localhost:8888/notebooks/issuer.ipynb) from Alice's perspective.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)

In [5]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [6]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [7]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Handle Credentials
Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
Handle Credentials
Credential exchange 4732cdb9-ccb7-40e4-88c5-c652fef1d6c6, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.

In [8]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Propose Credential (Optional)

If unhappy with the attributes being offered (e.g. there is a mistake), the holder can send a proposal to the issuer for an alternative credential. The holder can also initiate the protocol with a proposal to the issuer, this is left out of scope in the tutorial, but should be possible to edit to achieve this if interested.

Note the proposal will create a new credential exchange record, the tutorial currently does not progress this to issued state but instead continues with the previous credential_exchange.

**Arguments (These can all be found in a credential exchange record)**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [11]:
connection_id = cred_record['connection_id']
schema_id = cred_record['schema_id']
cred_def_id = cred_record['credential_definition_id']
proposed_attributes = [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]

proposal_record = await agent_controller.issuer.send_proposal(connection_id, schema_id, cred_def_id, proposed_attributes)
proposal_record_id = proposal_record['credential_exchange_id']
proposal_state = proposal_record['state']
proposal_role = proposal_record['role']
proposal_attributes = proposal_record['credential_proposal_dict']['credential_proposal']['attributes']
print(f"Proposal credential exchange record {proposal_record_id}, role: {proposal_role}, state: {proposal_state}")
print(f"Attributes: {attributes}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]}, 'connection_id': '49ec8c02-0616-40e1-b11a-55348a2a8f82', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'schema_name': 'open_mined_contributor', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Proposal credential exchange record ea8d1a1e-e1e3-48fd-b95b-8d0421313e7a, role: holder, state: proposal_sent
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
Handle Credentials
Credential exchange ea8d1a1e-e1e3-48fd-b95b-8d0421313e7a, role: holder, state: proposal_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software

## Request Credential from Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [13]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: request_sent
Handle Credentials
Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: request_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
Handle Credentials
Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: credential_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [14]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: credential_received


In [15]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: credential_acked
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
Credential exchange 2126baeb-114d-4da6-b2c2-a8cb326d7d2e, role: holder, state: credential_acked


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [16]:
response = await agent_controller.terminate()
